<a href="https://colab.research.google.com/github/rtajeong/Kyungnam_univ_M4/blob/main/lab_62_naver_movie_RNN_rev1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# GPU 런타임 설정을 한 후 실행한다.
!pip install konlpy

     |████████████████████████████████| 19.4MB 1.2MB/s 
     |████████████████████████████████| 460kB 48.7MB/s 
     |████████████████████████████████| 92kB 14.4MB/s 
  Found existing installation: tweepy 3.6.0
    Uninstalling tweepy-3.6.0:
      Successfully uninstalled tweepy-3.6.0
  Found existing installation: beautifulsoup4 4.6.3
    Uninstalling beautifulsoup4-4.6.3:
      Successfully uninstalled beautifulsoup4-4.6.3


In [2]:
# 패키지 설치
import konlpy
import pandas as pd
import numpy as np
from konlpy.tag import Twitter
from sklearn.feature_extraction.text import TfidfVectorizer
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from sklearn import model_selection, metrics

# 토큰 파서
def twitter_tokenizer(text):
    return Twitter().morphs(text)

In [3]:
twitter_tokenizer("Welcome to data science world!...한국말도 똑 같아요...")

/usr/local/lib/python3.6/dist-packages/konlpy/tag/_okt.py:16: UserWarning: "Twitter" has changed to "Okt" since KoNLPy v0.4.5.
  warn('"Twitter" has changed to "Okt" since KoNLPy v0.4.5.')


['Welcome',
 'to',
 'data',
 'science',
 'world',
 '!...',
 '한국말',
 '도',
 '똑',
 '같아요',
 '...']

In [4]:
!curl -L https://bit.ly/2X9Owwr -o ratings_train.txt
!curl -L https://bit.ly/2WuLd5I -o ratings_test.txt

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   152  100   152    0     0   4222      0 --:--:-- --:--:-- --:--:--  4222
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100 14.0M  100 14.0M    0     0  8067k      0  0:00:01  0:00:01 --:--:-- 13.3M
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   151  100   151    0     0   7947      0 --:--:-- --:--:-- --:--:--  7947
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100 4827k  100 4827k    0     0  4580k      0  0:00:01  0:00:01 --:--:--  167M


In [5]:
# 데이터 로드
df_train = pd.read_csv('ratings_train.txt', delimiter='\t', keep_default_na=False)
df_test = pd.read_csv('ratings_test.txt', delimiter='\t', keep_default_na=False)


In [6]:
df_train[:3]

,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0


In [7]:
df_train['document'].values == df_train['document'].to_numpy()

array([ True,  True,  True, ...,  True,  True,  True])

In [8]:
text_train, y_train = df_train['document'].to_numpy(), df_train['label'].to_numpy()
text_test, y_test = df_test['document'].to_numpy(), df_test['label'].to_numpy()

In [9]:
text_train.shape, y_train.shape, text_test.shape, y_test.shape

((150000,), (150000,), (50000,), (50000,))

In [10]:
# too much... -> let's take few of them
text_train, y_train = text_train[:2000], y_train[:2000]
text_test, y_test = text_test[:1000], y_test[:1000]

In [11]:
cv = TfidfVectorizer(tokenizer=twitter_tokenizer, max_features=3000)
X_train = cv.fit_transform(text_train)
X_test = cv.transform(text_test)

/usr/local/lib/python3.6/dist-packages/konlpy/tag/_okt.py:16: UserWarning: "Twitter" has changed to "Okt" since KoNLPy v0.4.5.
  warn('"Twitter" has changed to "Okt" since KoNLPy v0.4.5.')
/usr/local/lib/python3.6/dist-packages/konlpy/tag/_okt.py:16: UserWarning: "Twitter" has changed to "Okt" since KoNLPy v0.4.5.
  warn('"Twitter" has changed to "Okt" since KoNLPy v0.4.5.')


In [12]:
X_train.shape, y_train.shape, X_test.shape, y_test.shape

((2000, 3000), (2000,), (1000, 3000), (1000,))

In [13]:
X_train[0][-10:].toarray()

array([[0., 0., 0., ..., 0., 0., 0.]])

In [14]:
y_test[:3]

array([1, 0, 0])

In [15]:
cv.get_feature_names()[-5:]

['힘들건데', '힘들게', '힘들다', '힘들었네요', '힘들었음']

# 로지스틱 회귀

In [16]:
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression()
result = lr.fit(X_train,y_train)
feature_names = cv.get_feature_names()
print("Training : ", result.score(X_train, y_train))
print("Testing : ", result.score(X_test, y_test))

Training :  0.916
Testing :  0.771


# MLP

In [17]:
# use one-hot encoded target
from tensorflow.keras.utils import to_categorical
y_train_ohe = to_categorical(y_train)
y_test_ohe = to_categorical(y_test)
max_words = X_train.shape[1]
batch_size = 100
nb_epoch = 5

model = Sequential()
model.add(Dense(64, input_shape=(max_words,)))
model.add(Activation('relu'))
model.add(Dropout(0.2))
model.add(Dense(64))
model.add(Dropout(0.2))
model.add(Dense(2, activation='softmax'))
model.compile(loss='binary_crossentropy', optimizer='adam',
              metrics=['accuracy'])

model.fit(X_train.toarray(), y_train_ohe, epochs=nb_epoch, 
          batch_size=batch_size) 
print("Training : ", model.evaluate(X_train.toarray(), y_train_ohe))
print("Testing : ", model.evaluate(X_test.toarray(), y_test_ohe))

Epoch 1/5
20/20 [==============================] - 2s 17ms/step - loss: 0.6913 - accuracy: 0.5289
Epoch 2/5
20/20 [==============================] - 0s 2ms/step - loss: 0.6695 - accuracy: 0.7794
Epoch 3/5
20/20 [==============================] - 0s 2ms/step - loss: 0.5870 - accuracy: 0.8701
Epoch 4/5
20/20 [==============================] - 0s 2ms/step - loss: 0.4184 - accuracy: 0.9062
Epoch 5/5
63/63 [==============================] - 0s 2ms/step - loss: 0.1705 - accuracy: 0.9510
Training :  [0.17052431404590607, 0.9509999752044678]
32/32 [==============================] - 0s 2ms/step - loss: 0.5006 - accuracy: 0.7620
Testing :  [0.5005531311035156, 0.7620000243186951]


In [18]:
# use binary target

max_words = X_train.shape[1]
batch_size = 100
nb_epoch = 5

model = Sequential()
model.add(Dense(64, input_shape=(max_words,)))
model.add(Activation('relu'))
model.add(Dropout(0.2))
model.add(Dense(64))
model.add(Dropout(0.2))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam',
              metrics=['accuracy'])

model.fit(X_train.toarray(), y_train, epochs=nb_epoch, 
          batch_size=batch_size) 
print("Training : ", model.evaluate(X_train.toarray(), y_train))
print("Testing : ", model.evaluate(X_test.toarray(), y_test))

Epoch 1/5
20/20 [==============================] - 0s 2ms/step - loss: 0.6922 - accuracy: 0.5317
Epoch 2/5
20/20 [==============================] - 0s 2ms/step - loss: 0.6668 - accuracy: 0.7496
Epoch 3/5
20/20 [==============================] - 0s 2ms/step - loss: 0.5784 - accuracy: 0.8557
Epoch 4/5
20/20 [==============================] - 0s 2ms/step - loss: 0.4025 - accuracy: 0.9064
Epoch 5/5
63/63 [==============================] - 0s 2ms/step - loss: 0.1617 - accuracy: 0.9600
Training :  [0.16168324649333954, 0.9599999785423279]
32/32 [==============================] - 0s 2ms/step - loss: 0.5045 - accuracy: 0.7590
Testing :  [0.5044946074485779, 0.7590000033378601]


# RNN

In [19]:
# just for checking
X_train.A[0] == X_train.toarray()[0]

array([ True,  True,  True, ...,  True,  True,  True])

In [20]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((2000, 3000), (1000, 3000), (2000,), (1000,))

In [21]:
# RNN 학습을 위한 데이터 재배열
X_train_rnn = X_train.A.reshape((X_train.shape[0], 1, X_train.shape[1]))
X_test_rnn = X_test.A.reshape((X_test.shape[0], 1, X_test.shape[1]))

print(X_train_rnn.shape, X_test_rnn.shape)

(2000, 1, 3000) (1000, 1, 3000)


In [22]:
from keras.models import Sequential
from keras.layers import SimpleRNN, Dense

model = Sequential()
model.add(SimpleRNN(128, 
                    input_shape=(X_train_rnn.shape[1], X_train_rnn.shape[2]), 
                    return_sequences=True))
# return_sequences: return the last output in the output sequence, or the full sequence
# By this, it is possible to access the hidden state output for each input time step.
model.add(Activation('relu'))
model.add(Dropout(0.2))
model.add(SimpleRNN(128))
model.add(Dropout(0.2))
model.add(Dense(2, activation="softmax"))
model.compile(loss='categorical_crossentropy', 
              optimizer='adam', metrics=['accuracy'])   

In [ ]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
simple_rnn (SimpleRNN)       (None, 1, 128)            400512    
_________________________________________________________________
activation_2 (Activation)    (None, 1, 128)            0         
_________________________________________________________________
dropout_4 (Dropout)          (None, 1, 128)            0         
_________________________________________________________________
simple_rnn_1 (SimpleRNN)     (None, 128)               32896     
_________________________________________________________________
dropout_5 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_6 (Dense)              (None, 2)                 258       
Total params: 433,666
Trainable params: 433,666
Non-trainable params: 0
________________________________________________

In [23]:
model.fit(X_train_rnn, y_train_ohe, batch_size = 100,
          epochs=nb_epoch)

Epoch 1/5
20/20 [==============================] - 1s 4ms/step - loss: 0.6893 - accuracy: 0.5567
Epoch 2/5
20/20 [==============================] - 0s 4ms/step - loss: 0.6144 - accuracy: 0.8263
Epoch 3/5
20/20 [==============================] - 0s 4ms/step - loss: 0.3799 - accuracy: 0.9060
Epoch 4/5
20/20 [==============================] - 0s 4ms/step - loss: 0.1770 - accuracy: 0.9485
Epoch 5/5
20/20 [==============================] - 0s 4ms/step - loss: 0.0783 - accuracy: 0.9842


In [24]:
y_pred = np.argmax(model.predict(X_test_rnn), axis=1)
print("accuracy score: ", metrics.accuracy_score(y_test, y_pred))
print("Training : ", model.evaluate(X_train_rnn, y_train_ohe))
print("Testing : ", model.evaluate(X_test_rnn, y_test_ohe))

accuracy score:  0.762
63/63 [==============================] - 0s 3ms/step - loss: 0.0495 - accuracy: 0.9895
Training :  [0.04952521622180939, 0.9894999861717224]
32/32 [==============================] - 0s 2ms/step - loss: 0.6084 - accuracy: 0.7620
Testing :  [0.6084091067314148, 0.7620000243186951]


# LSTM
- https://colah.github.io/posts/2015-08-Understanding-LSTMs/

In [25]:
from keras.layers import LSTM
model = Sequential()
model.add(LSTM(128, 
               input_shape=(X_train_rnn.shape[1], X_train_rnn.shape[2]), 
               return_sequences=True))
model.add(Activation('relu'))
model.add(Dropout(0.2))
model.add(LSTM(128))
model.add(Dropout(0.2))
model.add(Dense(2, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [28]:
model.fit(X_train_rnn, y_train_ohe, batch_size = 100,
          epochs=nb_epoch)

Epoch 1/5
20/20 [==============================] - 0s 5ms/step - loss: 0.1752 - accuracy: 0.9400
Epoch 2/5
20/20 [==============================] - 0s 5ms/step - loss: 0.1133 - accuracy: 0.9650
Epoch 3/5
20/20 [==============================] - 0s 5ms/step - loss: 0.0790 - accuracy: 0.9795
Epoch 4/5
20/20 [==============================] - 0s 5ms/step - loss: 0.0571 - accuracy: 0.9830
Epoch 5/5
20/20 [==============================] - 0s 5ms/step - loss: 0.0457 - accuracy: 0.9865


In [29]:
y_pred = np.argmax(model.predict(X_test_rnn), axis=1)
print("accuracy score: ", metrics.accuracy_score(y_test, y_pred))
print("Training : ", model.evaluate(X_train_rnn, y_train_ohe))
print("Testing : ", model.evaluate(X_test_rnn, y_test_ohe))

accuracy score:  0.757
63/63 [==============================] - 0s 3ms/step - loss: 0.0343 - accuracy: 0.9905
Training :  [0.034298501908779144, 0.9904999732971191]
32/32 [==============================] - 0s 3ms/step - loss: 0.8071 - accuracy: 0.7570
Testing :  [0.807087779045105, 0.7570000290870667]


# GRU

In [30]:
from keras.layers import GRU
model = Sequential()
model.add(GRU(128, input_shape=(X_train_rnn.shape[1], X_train_rnn.shape[2]), return_sequences=True))
model.add(Activation('relu'))
model.add(Dropout(0.2))
model.add(GRU(128))
model.add(Dropout(0.2))
model.add(Dense(2, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [31]:
model.fit(X_train_rnn, y_train_ohe, batch_size = 100,
          epochs=nb_epoch)

Epoch 1/5
20/20 [==============================] - 3s 4ms/step - loss: 0.6923 - accuracy: 0.5225
Epoch 2/5
20/20 [==============================] - 0s 4ms/step - loss: 0.6752 - accuracy: 0.7333
Epoch 3/5
20/20 [==============================] - 0s 4ms/step - loss: 0.5549 - accuracy: 0.8871
Epoch 4/5
20/20 [==============================] - 0s 4ms/step - loss: 0.3117 - accuracy: 0.9079
Epoch 5/5
20/20 [==============================] - 0s 4ms/step - loss: 0.1642 - accuracy: 0.9418


In [32]:
y_pred = np.argmax(model.predict(X_test_rnn), axis=1)
print("accuracy score: ", metrics.accuracy_score(y_test, y_pred))
print("Training : ", model.evaluate(X_train_rnn, y_train_ohe))
print("Testing : ", model.evaluate(X_test_rnn, y_test_ohe))

accuracy score:  0.757
63/63 [==============================] - 1s 3ms/step - loss: 0.1004 - accuracy: 0.9690
Training :  [0.10035163909196854, 0.968999981880188]
32/32 [==============================] - 0s 3ms/step - loss: 0.5828 - accuracy: 0.7570
Testing :  [0.5828368663787842, 0.7570000290870667]
